In [7]:
import os
import sys
from gurobipy import *
import pandas as pd
module_path = os.path.abspath("..")
sys.path.append(module_path)
import itertools
import datetime
import plotly.express as px

from module.modele1 import init_model,Taches
from module.utils1 import (
    read_sillon,
    init_t_a,
    init_t_d,
    init_dict_correspondance,
    base_time
)

In [2]:
# Charger le fichier Excel
file = "instance_WPY_realiste_jalon1.xlsx"

# Lire les feuilles "Sillons arrivée" et "Sillons départ"
df_sillons_arr, df_sillons_dep = read_sillon(file)

# Dictionnaires pour stocker les temps d'arrivée et de départ en minutes :
# Traitement des arrivées
t_a = init_t_a(df_sillons_arr, print_bool=True)

t_d = init_t_d(df_sillons_dep, print_bool=True)

# Vérification finale
print("Vérification du contenu final de t_a :")
for key, value in t_a.items():
    print(f"Train {key}: {value} min")

print("Vérification du contenu final de t_d :")
for key, value in t_d.items():
    print(f"Train {key}: {value} min")

Train 412202_08 : JARR = 2022-08-08, minutes écoulées = 1248
Train 431018_08 : JARR = 2022-08-08, minutes écoulées = 801
Train 431246_08 : JARR = 2022-08-08, minutes écoulées = 664
Train 44951_08 : JARR = 2022-08-08, minutes écoulées = 1130
Train 450237_08 : JARR = 2022-08-08, minutes écoulées = 707
Train 489016_08 : JARR = 2022-08-08, minutes écoulées = 1415
Train 54234_08 : JARR = 2022-08-08, minutes écoulées = 1321
Train 55390_08 : JARR = 2022-08-08, minutes écoulées = 1429
Train 56135_08 : JARR = 2022-08-08, minutes écoulées = 1288
Train 412023_09 : JARR = 2022-08-09, minutes écoulées = 1567
Train 41264_09 : JARR = 2022-08-09, minutes écoulées = 2793
Train 431802_09 : JARR = 2022-08-09, minutes écoulées = 1620
Train 44222_09 : JARR = 2022-08-09, minutes écoulées = 2533
Train 44250_09 : JARR = 2022-08-09, minutes écoulées = 2058
Train 44865_09 : JARR = 2022-08-09, minutes écoulées = 2416
Train 450237_09 : JARR = 2022-08-09, minutes écoulées = 2147
Tra

In [3]:
df_correspondance = pd.read_excel(file, sheet_name="Correspondances")

D = init_dict_correspondance(df_correspondance)

In [4]:
model,t_arr,t_dep = init_model(
    liste_id_train_arrivee = t_a.keys(),
    t_a = t_a,
    liste_id_train_depart = t_d.keys(),
    t_d = t_d,
    dict_correspondances = D,
    file=file,
    id_file = 1
)


Set parameter Username
Set parameter LicenseID to value 2617576
Academic license - for non-commercial use only - expires 2026-02-03


In [5]:
model.optimize()

if model.status == GRB.INFEASIBLE:
    print("Le modèle n'a pas de solution")
elif model.status == GRB.UNBOUNDED:
    print("Le modèle est non borné")
else:
    print("Ca marche Bébou")
    for (m, n), var in t_arr.items():
        print(f"Tâche {m} - Train {n} : {var.x}")
    for (m, n), var in t_dep.items():
        print(f"Tâche {m} - Train {n} : {var.x}")

Ca marche Bébou
Tâche 1 - Train 412202_08 : 1248.0
Tâche 1 - Train 431018_08 : 801.0
Tâche 1 - Train 431246_08 : 664.0
Tâche 1 - Train 44951_08 : 1130.0
Tâche 1 - Train 450237_08 : 707.0
Tâche 1 - Train 489016_08 : 1415.0
Tâche 1 - Train 54234_08 : 1321.0
Tâche 1 - Train 55390_08 : 1429.0
Tâche 1 - Train 56135_08 : 1288.0
Tâche 1 - Train 412023_09 : 1567.0
Tâche 1 - Train 41264_09 : 2793.0
Tâche 1 - Train 431802_09 : 1620.0
Tâche 1 - Train 44222_09 : 2533.0
Tâche 1 - Train 44250_09 : 2058.0
Tâche 1 - Train 44865_09 : 2416.0
Tâche 1 - Train 450237_09 : 2147.0
Tâche 1 - Train 471003_09 : 2203.0
Tâche 1 - Train 47262_09 : 1982.0
Tâche 1 - Train 489026_09 : 2011.0
Tâche 1 - Train 54234_09 : 2761.0
Tâche 1 - Train 55390_09 : 2869.0
Tâche 1 - Train 55398_09 : 1680.0
Tâche 1 - Train 56135_09 : 2728.0
Tâche 1 - Train 56730_09 : 1592.0
Tâche 1 - Train 58706_09 : 2715.0
Tâche 1 - Train 412023_10 : 3007.0
Tâche 1 - Train 412202_10 : 4128.0
Tâche 1 - Train 41264_10 : 4233.0
Tâche 1 - Train 431020_

In [9]:
# Liste ordonnée des machines
ORDERED_MACHINES = ["arr_1", "arr_2", "arr_3",
                        "dep_1", "dep_2", "dep_3", "dep_4"]

# Données fournies sous forme de liste de dictionnaires
tasks = [
        {"Train": n_arr,
         "Start": base_time(1) + datetime.timedelta(minutes=var_arr.X),
         "Finish": base_time(1) + datetime.timedelta(minutes=var_arr.X + Taches.T_ARR[m_arr]),
         "Machine": f"arr_{m_arr}",
         "Tâches": f"arr_{m_arr}"}
        for (m_arr, n_arr), var_arr in t_arr.items()
    ] + [
        {"Train": n_dep,
         "Start": base_time(1) + datetime.timedelta(minutes=var_dep.X),
         "Finish": base_time(1) + datetime.timedelta(minutes=var_dep.X + Taches.T_DEP[m_dep]),
         "Machine": f"dep_{m_dep}",
         "Tâches": f"dep_{m_dep}"}
        for (m_dep, n_dep), var_dep in t_dep.items()
    ]

# Construction du DataFrame pour la visualisation
gantt_df = pd.DataFrame(tasks)

# Regroupement des ressources par machine
resource_per_machine = {}
for task in tasks:
    resource_per_machine.setdefault(
        task["Machine"], set()).add(task["Tâches"])

sorted_resources = list(itertools.chain.from_iterable(
    [sorted(resource_per_machine[machine]) for machine in ORDERED_MACHINES]
))

fig = px.timeline(gantt_df, x_start="Start",
                    x_end="Finish", y="Tâches", color="Train")
fig.update_layout(xaxis=dict(title='Temps',
                    tickformat='%d/%m/%y %H:%M'))
fig.update_yaxes(categoryorder="array",
                    categoryarray=sorted_resources[::-1])
fig.show()